## Notebook to visualize the viewer functionality


In [ ]:
from notebook_imports import *

import time
import torch
from torch.utils.data import DataLoader

from pyrad.data.dataloader import TrainDataloader
from pyrad.data.image_dataset import ImageDataset, PanopticImageDataset
from pyrad.data.image_sampler import CacheImageSampler
from pyrad.data.pixel_sampler import PixelSampler
from pyrad.data.utils import DatasetInputs, get_dataset_inputs_from_dataset_config
from pyrad.graphs.modules.ray_generator import RayGenerator
from pyrad.graphs.modules.scene_colliders import SceneBoundsCollider, AABBBoxCollider
from pyrad.cameras.cameras import get_camera
from pyrad.cameras.rays import RayBundle
from pyrad.utils.io import get_absolute_path
from pyrad.utils.plotly import get_line_segments_from_lines
from pyrad.cameras.cameras import get_camera_model
from pyrad.utils.misc import get_dict_to_torch, instantiate_from_dict_config
from pyrad.engine.trainer import Trainer
from pyrad.utils.config import Config, setup_config

from hydra import compose, initialize
from omegaconf import open_dict
import pprint
from tqdm import tqdm
import random

import numpy as np

import umsgpack
from pyrad.viewer.server.utils import get_chunks, get_intrinsics_matrix_and_camera_to_world_h

from pyrad.viewer.server import viewer_utils
from pyrad.viewer.server.visualizer import Viewer
from pyrad.viewer.server.utils import get_camera_from_vis

In [ ]:
# connect to the viewer
vis = Viewer(zmq_url="tcp://127.0.0.1:6000")

In [ ]:
# draw a red box
viewer_utils.show_box_test(vis)

In [ ]:
# move the camera in a spiral path
camera = get_camera_from_vis(vis)
camera_path = get_spiral_path(camera, steps=120, radius=0.5)
seconds = 5
fps = len(camera_path.cameras) / seconds
spf = 1 / fps
for cam in camera_path.cameras:
    viewer_utils.set_camera(vis, cam)
    time.sleep(spf)